## Chapter 15. More About Typing

### Overloaded Signatures

In [1]:
# Example 15-1. mysum.py: definition of the sum function with overloaded signatures

import functools
import operator
from collections.abc import Iterable
from typing import overload, Union, TypeVar

T = TypeVar('T')
S = TypeVar('S')  # <1>

@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...  # <2>
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...  # <3>
def sum(it, /, start=0):  # <4>
    return functools.reduce(operator.add, it, start)

In [2]:
help(sum)

Help on function sum in module __main__:

sum(it, /, start=0)



In [ ]:
# Example 15-3. mymax.py: top of the module, with imports, definitions, and overloads

from collections.abc import Callable, Iterable
from typing import Protocol, Any, TypeVar, overload, Union

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

T = TypeVar('T')
LT = TypeVar('LT', bound=SupportsLessThan)
DT = TypeVar('DT')

MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT:      #1
    ...
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T:   #2
    ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT:            #3
    ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T:       #4
    ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...,                   #5
        default: DT) -> Union[LT, DT]:
    ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT],             #6
        default: DT) -> Union[T, DT]:
    ...

# Example 15-2. mymax.py: Python rewrite of max function

# imports and definitions omitted, see next listing
MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'

# overloaded type hints omitted, see next listing

def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = first
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

In [ ]:
# Arguments implementing SupprtsLessThan, but key and default not provided: #1, #3
print(max(1, 2, -3))  # returns 1
print(max(['Go', 'Python', 'Rust']))  # returns 'Rust'

# Argument key provided, but no default: #2, #4
print(max(1, 2, -3, key=abs))  # returns -3
print(max(['Go', 'Python', 'Rust'], key=len))  # returns 'Python'

# Argument default provided, but no key: #5
print(max([1, 2, -3], default=0))  # returns 2
print(max([], default=None))  # returns None

# Arguments key and default provided: #6
print(max([1, 2, -3], key=abs, default=None))  # returns -3
print(max([], key=abs, default=None))  # returns None

# max([None, None])  # Error

2
Rust
-3
Python
2
None
-3
None


TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

### TypedDict

In [28]:
# Example 15-4. books.py: the BookDict definition

import json
from typing import TypedDict

class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int


# Example 15-8. books.py: to_xml function

AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'

def to_xml(book: BookDict) -> str:  # <1>
    elements: list[str] = []  # <2>
    for key, value in book.items():
        if isinstance(value, list):  # <3>
            elements.extend(
                AUTHOR_ELEMENT.format(n) for n in value)  # <4>
        else:
            tag = key.upper()
            elements.append(f'<{tag}>{value}</{tag}>')
    xml = '\n\t'.join(elements)
    return f'<BOOK>\n\t{xml}\n</BOOK>'

# Example 15-9. books_any.py: from_json function

def from_json(data: str) -> BookDict:
    whatever = json.loads(data)  # <1>
    return whatever  # <2>


# Example 15-10. books.py: from_json function with variable annotation

def from_json(data: str) -> BookDict:
    whatever: BookDict = json.loads(data)  # <1>
    return whatever  # <2>

In [24]:
# Example 15-5. Using a BookDict, but not quite as intended

pp = BookDict(title='Programming Pearls', authors='Jon Bentley',
              isbn='0201657880', pagecount=256)
print(pp)
print(type(pp))
# print(pp.title)  # Error since BookDict is a plain dict
print(pp['title'])
print(BookDict.__annotations__)

{'title': 'Programming Pearls', 'authors': 'Jon Bentley', 'isbn': '0201657880', 'pagecount': 256}
<class 'dict'>
Programming Pearls
{'isbn': <class 'str'>, 'title': <class 'str'>, 'authors': list[str], 'pagecount': <class 'int'>}


In [27]:
# Example 15-6. demo_books.py: legal and illegal operations on a BookDict

#from books import BookDict
from typing import TYPE_CHECKING

def demo() -> None:  # <1>
    book = BookDict(  # <2>
        isbn='0134757599',
        title='Refactoring, 2e',
        authors=['Martin Fowler', 'Kent Beck'],
        pagecount=478
    )
    authors = book['authors'] # <3>
    if TYPE_CHECKING:  # <4>
        reveal_type(authors)  # <5>
    authors = 'Bob'  # Error
    book['weight'] = 4.2 # Error
    del book['title'] # Error

demo()

In [29]:
# Example 15-11. demo_not_book.py: from_json returns an invalid BookDict, and to_xml accepts it

# from books import to_xml, from_json
from typing import TYPE_CHECKING

def demo() -> None:
    NOT_BOOK_JSON = """
        {"title": "Andromeda Strain",
         "flavor": "pistachio",
         "authors": true}
    """
    not_book = from_json(NOT_BOOK_JSON)  # <1>
    if TYPE_CHECKING:  # <2>
        reveal_type(not_book)
        reveal_type(not_book['authors'])

    print(not_book)  # <3>
    print(not_book['flavor'])  # <4>

    xml = to_xml(not_book)  # <5>
    print(xml)  # <6>


demo()

{'title': 'Andromeda Strain', 'flavor': 'pistachio', 'authors': True}
pistachio
<BOOK>
	<TITLE>Andromeda Strain</TITLE>
	<FLAVOR>pistachio</FLAVOR>
	<AUTHORS>True</AUTHORS>
</BOOK>
